# 提案手法まとめ  

***
## LM(Logarithmic Median)法

先行研究の区間AHP手法では，重要度の中心値が大きくなるほど幅が小さく推定される傾向がある．  
この傾向はセッティングBでの推定精度を下げるため，上記の性質を緩和すべく，対数空間上での最適化を考える．  
これは対数空間上でのメディアンを考えることに相当し，外れ値の影響を受けにくくなると予想される．

$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
\sum_{i \in N} \sum_{j < i} \left| \ln \left( a_{ij} \right) - \ln \left( \frac{w_i}{w_j} \right) \right| \\
\end{align*}
$$

上式は非線形計画問題であるが，以下のように文字を置き換えることによって，線形計画問題に帰着する．

$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
\sum_{i \in N} \sum_{j < i} U_{ij}, \\
\text{subject to} ~~ & ~~
\ln \left( a_{ij} \right) - u_i + u_j \leq U_{ij}, &&
i \in N, ~~ & j<i \\ & ~~
u_i - u_j -\ln \left( a_{ij} \right) \leq U_{ij}, &&
i \in N, ~~ & j<i \\ & ~~
u_i \in \mathbb{R}.
\end{align*}
$$


$$where ~~ u_i = \ln \left( w_i \right), ~ i \in N$$

ここで，求まった最適解から
$$w_i = \lambda \exp \left( u_i \right)$$
として重みを計算することができるが，この際，総和が１になるように定める．

<div style="page-break-before:always"></div>

***
## eMMRw, gMMRw, lMMRw (プログラム上はMMRと表記)

##### 以下のPhase1～3を$k=1...n$でループ:

**Phase 1**  
$k$番目以外の重要度比率を固定  
* PCMの$k$行$k$列を除く  
* 上記の行列にEV, GM, LM を適用し， $w_{ki} ~ (i \in N\backslash k)$ を求める（総和は１）
* $w_{ki} ~ (i \in N\backslash k)$ の和を $0<\mu_k<1$ 倍することで， $k$ 番目の区間重要度中心は $1-\mu_k$ となる．　　

上記の操作によって，$k$番目の重みに自由度を残しつつ，その他の重みの比を固定することができる．  
このような弱モデルが$n$個生成される．

**Phase 2**  
以下の線形最適化問題を解き，最適値を $d_{\overline{k}}^{*}$ とする：  
$$
\begin{align*}

\operatorname{minimize} ~~ & ~~
d_{\overline{k}} = \sum_{j \in N \backslash k} l_{kj}, \\

\text{subject to} ~~ & ~~
a_{ij} \left( \mu_k w_{kj} - l_{kj} \right) \leq \mu_k w_{ki} + l_{ki}, &&
i \neq j, k, ~~ & j \neq k \\ & ~~

a_{kj} \left( \mu_k w_{kj} - l_{kj} \right) \leq 1-\mu_k + l_{kk}, &&
j \neq k \\ & ~~

a_{ik} \left\{ \left(1- \mu_k \right) - l_{kk} \right\} \leq \mu_k w_{ki} + l_{ki}, &&
i \neq k \\ & ~~

\left( 1 - \mu_k \right) + l_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( \mu_k w_{ki} + l_{ki} \right) + \mu_k w_{kj} - l_{kj} \geq 1, &&
i \in N \backslash k \\ & ~~

\left( 1 - \mu_k \right) - l_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\}} \left( \mu_k w_{ki} - l_{ki} \right) + \mu_k w_{kj} + l_{kj} \leq 1, &&
i \in N \backslash k \\ & ~~


\sum_{i \in N \backslash k} \left( \mu_k w_{ki} + l_{ki} \right) + \left( 1 - \mu_k \right) - l_{kk} \geq 1 \\ & ~~
\sum_{i \in N \backslash k} \left( \mu_k w_{ki} - l_{ki} \right) + \left( 1 - \mu_k \right) + l_{kk} \leq 1 \\ & ~~

\mu_k w_{ki} - l_{ki} \geq \epsilon, &&
i \in N \backslash k ~~ & \\ & ~~

\left( 1 - \mu_k \right) - l_{kk} \geq \epsilon \\ & ~~


\end{align*}
$$

<div style="page-break-before:always"></div>


**Phase 3**  
以下の線形最適化問題を解き，最適値を $\mu_{k}^*, ~ l_{ki}^{*} ~(i \in N)$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
l_{kk} \\
\text{subject to} ~~ & ~~
a_{ij} \left( \mu_k w_{kj} - l_{kj} \right) \leq \mu_k w_{ki} + l_{ki}, &&
i \neq j, k, ~~ & j \neq k \\ & ~~

a_{kj} \left( \mu_k w_{kj} - l_{kj} \right) \leq 1-\mu_k + l_{kk}, &&
j \neq k \\ & ~~

a_{ik} \left\{ \left(1- \mu_k \right) - l_{kk} \right\} \leq \mu_k w_{ki} + l_{ki}, &&
i \neq k \\ & ~~

\left( 1 - \mu_k \right) + l_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( \mu_k w_{ki} + l_{ki} \right) + \mu w_{kj} - l_{kj} \geq 1, &&
i \in N \backslash k \\ & ~~

\left( 1 - \mu_k \right) - l_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( \mu_k w_{ki} - l_{ki} \right) + \mu_k w_{kj} + l_{kj} \leq 1, &&
i \in N \backslash k \\ & ~~

\sum_{i \in N \backslash k} \left( \mu_k w_{ki} + l_{ki} \right) + \left( 1 - \mu_k \right) - l_{kk} \geq 1 \\ & ~~

\sum_{i \in N \backslash k} \left( \mu_k w_{ki} - l_{ki} \right) + \left( 1 - \mu_k \right) + l_{kk} \leq 1 \\ & ~~

\sum_{j \in N \backslash k} l_{kj} = d_{\overline{k}}^{*}\\ & ~~

\mu_k w_{ki} - l_{ki} \geq \epsilon, &&
i \in N \backslash k ~~ & \\ & ~~

\left( 1 - \mu_k \right) - l_{kk} \geq \epsilon \\ & ~~


\end{align*}
$$

**Phase 4**  
Phase 3 で導かれた結果は$n$個の弱モデル分だけ考えられるため．以下のように結果を統合する：   
$$\left\{\begin{array}{l}w_i(k)=\left[\mu_{k}^* w_{ki}-l_{ki}^{*},\:\: \mu_{k}^* w_{ki}+l_{ki}^{*}\right], \quad i \in N\backslash k \\ w_i(k)=\left[\left(1-\mu_{k}^*\right)-l_{ki}^{*},\:\:\left(1-\mu_{k}^*\right)+l_{ki}^{*}\right], \quad i=k\end{array}\right.$$
  

$$\overline{w}_i^L=\min _{k \in N} w_i^L(k) \quad \overline{w}_i^U=\max _{k \in N} w_i^U(k)$$  
  
ここで，$\overline{w}_i^L$ と $\overline{w}_i^U$ の中心 $\overline{w}_i^C$の総和が1となるように正規化する：
  
$$\overline{\overline{w}}_i^L=\frac{\overline{w}_i^L}{\sum_{i \in N} \frac{\overline{w}_i^L+\overline{w}_i^U}{2}} \quad \overline{\overline{w}}_i^U=\frac{\overline{w}_i^U}{\sum_{i \in N} \frac{\overline{w}_i^L+\overline{w}_i^U}{2}}$$

<div style="page-break-before:always"></div>

***
## eMMRw/c, gMMRw/c, lMMRw/c (プログラム上はMMR-kaiと表記)

eMMRwなどでは，損失関数に区間重要度幅$_{W}$を用いた．  
損失関数に区間重要度幅と区間重要度中心の比率を用いた手法も考えられ，これを$_{W/C}$と表記することとする．

##### Phase1～3を$k=1...n$でループ:  
**Phase 1**  
上記手法と同様

**Phase 2【非線形】**  
各 $k$ に対して、以下の最適化問題を解き、最適値を $d_{\overline{k}}^{*}$ とする：  
（目的関数を重みと幅の比で扱うために，これまでの各式に対して，$\dfrac{1}{\mu_k}$ 倍している）  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~

d_{\overline{k}} = \sum_{j \in N \backslash k} \frac{l_{kj}}{\mu_k}, \\
\text{subject to} ~~ & ~~

a_{ij} \left( w_{kj} - \frac{l_{kj}}{\mu_k} \right) \leq w_{ki} + \frac{l_{ki}}{\mu_k}, &&
i \neq j, k, ~~ & j \neq k \\ & ~~

a_{kj} \left( w_{kj} - \frac{l_{kj}}{\mu_k} \right) \leq \frac{1-\mu_k}{\mu_k} + \frac{l_{kk}}{\mu_k}, &&
j \neq k \\ & ~~

a_{ik} \left(\frac{1-\mu_k}{\mu_k} - \frac{l_{kk}}{\mu_k} \right) \leq w_{ki} + \frac{l_{ki}}{\mu_k}, &&
i \neq k \\ & ~~

\frac{1-\mu_k}{\mu_k} + \frac{l_{kk}}{\mu_k} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( w_{ki} + \frac{l_{ki}}{\mu_k} \right) + w_{kj} - \frac{l_{kj}}{\mu_k} \geq \frac{1}{\mu_k}, &&
i \in N \backslash k \\ & ~~

\frac{1-\mu_k}{\mu_k} - \frac{l_{kk}}{\mu_k} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( w_{ki} - \frac{l_{ki}}{\mu_k} \right) + w_{kj} + \frac{l_{kj}}{\mu_k} \leq \frac{1}{\mu_k}, &&
i \in N \backslash k \\ & ~~

\sum_{i \in N \backslash k} \left( w_{ki} + \frac{l_{ki}}{\mu_k} \right) + \frac{1-\mu_k}{\mu_k} - \frac{l_{kk}}{\mu_k} \geq \frac{1}{\mu_k} \\ & ~~

\sum_{i \in N \backslash k} \left( w_{ki} - \frac{l_{ki}}{\mu_k} \right) + \frac{1-\mu_k}{\mu_k} + \frac{l_{kk}}{\mu_k} \leq \frac{1}{\mu_k} \\ & ~~

w_{ki} - \frac{l_{ki}}{\mu_k} \geq \frac{\epsilon}{\mu_k}, &&
i \in N, ~~ & i \neq k \\ & ~~

\frac{1-\mu_k}{\mu_k} - \frac{l_{kk}}{\mu_k} \geq \frac{\epsilon}{\mu_k} \\ & ~~


\end{align*}
$$

<div style="page-break-before:always"></div>

ただし，この問題は非線形最適化問題となるため，  
$\dfrac{l_{ki}}{\mu_k} = L_{ki}, ~~ \dfrac{1}{\mu_k} = t_k$  
と置き換えることで線形最適化問題に持ち込む：

**Phase 2' 【線形】**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $d_{\overline{k}}^{*}$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~

d_{\overline{k}} = \sum_{j \in N \backslash k} L_{kj}, \\
\text{subject to} ~~ & ~~

a_{ij} \left( w_{kj} - L_{kj} \right) \leq w_{ki} + L_{ki}, &&
i \neq j, k, ~~ & j \neq k \\ & ~~

a_{kj} \left( w_{kj} - L_{kj} \right) \leq t_k-1 + L_{kk}, &&
j \neq k \\ & ~~

a_{ik} \left\{ \left( t_k -1 \right) - L_{kk} \right\} \leq w_{ki} + L_{ki}, &&
i \neq k \\ & ~~

\left( t_k-1 \right) + L_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( w_{ki} + L_{ki} \right) + w_{kj} - L_{kj} \geq t_k, &&
i \in N \backslash k \\ & ~~

\left( t_k-1 \right) - L_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( w_{ki} - L_{ki} \right) + w_{kj} + L_{kj} \leq t_k, &&
i \in N \backslash k \\ & ~~

\sum_{i \in N \backslash k} \left( w_{ki} + L_{ki} \right) + \left( t_k-1 \right) - L_{kk} \geq t_k \\ & ~~

\sum_{i \in N \backslash k} \left( w_{ki} - L_{ki} \right) + \left( t_k-1 \right) + L_{kk} \leq t_k \\ & ~~

w_{ki} - L_{ki} \geq \epsilon , &&
i \in N, ~~ & i \neq k \\ & ~~

\left( t_k-1 \right) - L_{kk} \geq \epsilon \\ & ~~


\end{align*}
$$

<div style="page-break-before:always"></div>


**Phase 3【非線形】**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $\mu_{k}^*, ~ l_{ki}^{*} ~(i \in N)$ とする：  
（目的関数を重みと幅の比で扱うために，これまでの各式に対して，$\dfrac{1}{1-\mu_k}$ 倍している）  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
\frac{l_{kk}}{1-\mu_k} \\
\text{subject to} ~~ & ~~

a_{ij} \left( \frac{\mu_k}{1-\mu_k} w_{kj} - \frac{l_{kj}}{1-\mu_k} \right) \leq \frac{\mu_k}{1-\mu_k} w_{ki} + \frac{l_{ki}}{1-\mu_k}, &&
i \neq j, k, ~~ & j \neq k \\ & ~~

a_{kj} \left( \frac{\mu_k}{1-\mu_k} w_{kj} - \frac{l_{kj}}{1-\mu_k} \right) \leq 1 + \frac{l_{kk}}{1-\mu_k}, &&
j \neq k \\ & ~~

a_{ik} \left( 1 - l_{kk} \right) \leq \frac{\mu_k}{1-\mu_k} w_{ki} + \frac{l_{ki}}{1-\mu_k}, &&
i \neq k \\ & ~~

1 + \frac{l_{kk}}{1-\mu_k} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( \frac{\mu_k}{1-\mu_k} w_{ki} + \frac{l_{ki}}{1-\mu_k} \right) + \frac{\mu_k}{1-\mu_k} w_{kj} - \frac{l_{kj}}{1-\mu_k} \geq \frac{1}{1-\mu_k}, &&
i \in N \backslash k \\ & ~~

1 - \frac{l_{kk}}{1-\mu_k} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left( \frac{\mu_k}{1-\mu_k} w_{ki} - \frac{l_{ki}}{1-\mu_k} \right) + \frac{\mu_k}{1-\mu_k} w_{kj} + \frac{l_{kj}}{1-\mu_k} \leq \frac{1}{1-\mu_k}, &&
i \in N \backslash k \\ & ~~

\sum_{i \in N \backslash k} \left( \frac{\mu_k}{1-\mu_k} w_{ki} + \frac{l_{ki}}{1-\mu_k} \right) + 1 - \frac{l_{kk}}{1-\mu_k} \geq \frac{1}{1-\mu_k} \\ & ~~

\sum_{i \in N \backslash k} \left( \frac{\mu_k}{1-\mu_k} w_{ki} - \frac{l_{ki}}{1-\mu_k} \right) + 1 + \frac{l_{kk}}{1-\mu_k} \leq \frac{1}{1-\mu_k} \\ & ~~

\sum_{j \in N \backslash k} \frac{l_{kj}}{1-\mu_k} = \frac{\mu_k}{1-\mu_k} d_{\overline{k}}^{*}\\ & ~~

\frac{\mu_k}{1-\mu_k} w_{ki} - \frac{l_{ki}}{1-\mu_k} \geq \epsilon, &&
i \in N \backslash k ~~ & \\ & ~~

1 - \frac{l_{kk}}{1-\mu_k} \geq \epsilon \\ & ~~


\end{align*}
$$

<div style="page-break-before:always"></div>

ただし，この問題は非線形最適化問題となるため，  
$\dfrac{l_{ki}}{1-\mu_k} = L'_{ki}, ~~ \dfrac{1}{1-\mu_k} = t_k'  ~(i \in N)$  
と置き換えることで線形最適化問題に持ち込む：

**Phase 3'【線形】**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $\mu_{k}^*, ~ L_{ki}^{'*} ~(i \in N)$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
L'_{kk} \\
\text{subject to} ~~ & ~~
a_{ij} \left\{ \left( t_k'-1 \right) w_{kj} - L'_{kj} \right\} \leq \left( t_k'-1 \right) w_{ki} + L'_{ki}, &&
i \neq j, k, ~~ & j \neq k \\ & ~~

a_{kj} \left\{ \left( t_k'-1 \right) w_{kj} - L'_{kj} \right\} \leq 1 + L'_{kk}, &&
j \neq k \\ & ~~

a_{ik} \left(1 - L'_{kk} \right) \leq \left( t_k'-1 \right) w_{ki} + L'_{ki}, &&
i \neq k \\ & ~~

1 + L'_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left\{ \left( t_k'-1 \right) w_{ki} + L'_{ki} \right\} + \left( t_k'-1 \right) w_{kj} - L'_{kj} \geq t_k', &&
i \in N \backslash k \\ & ~~

1 - L'_{kk} + \sum_{i \in N \backslash \left\{ j, k \right\} } \left\{ \left( t_k'-1 \right) w_{ki} - L'_{ki} \right\} + \left( t_k'-1 \right) w_{kj} + L'_{kj} \leq t_k', &&
i \in N \backslash k \\ & ~~

\sum_{i \in N \backslash k} \left\{ \left( t_k'-1 \right) w_{ki} + L'_{ki} \right\} + 1 - L'_{kk} \geq t_k' \\ & ~~

\sum_{i \in N \backslash k} \left\{ \left( t_k'-1 \right) w_{ki} - L'_{ki} \right\} + 1 + L'_{kk} \leq t_k' \\ & ~~

\sum_{j \in N \backslash k} L'_{kj} = \left( t_k'-1 \right) d_{\overline{k}}^{*}\\ & ~~

\left( t_k'-1 \right) w_{ki} - L'_{ki} \geq \epsilon, &&
i \in N \backslash k ~~ & \\ & ~~

1 - L'_{kk} \geq \epsilon \\ & ~~


\end{align*}


この最適値から，以下のようにして目的の値を求めることができる．  
$$
\mu_{k}^{*}=1-\dfrac{1}{{t'}_{k}^{*}}, ~~ l_{ki}^{*}=\dfrac{L_{ki}^{'*}}{{t'}_{k}^{*}} ~~ (i \in N)
$$


**Phase 4**  
eMMRw, gMMRw, lMMRwと同様

***
## eAMRw, gAMRw, lAMRw (プログラム上はAMRと表記)

eMMRw. gMMRw, lMMRwの**Phase 4**を以下のように変更する：  
**Phase 4**  
Phase 3 で導かれた結果は$n$個の弱モデル分だけ考えられるため．以下のように結果を統合する：   
$$\left\{\begin{array}{l}w_i(k)=\left[\mu_{k}^* w_{ki}-l_{ki}^{*},\:\: \mu_{k}^* w_{ki}+l_{ki}^{*}\right], \quad i \in N\backslash k \\ w_i(k)=\left[\left(1-\mu_{k}^*\right)-l_{ki}^{*},\:\:\left(1-\mu_{k}^*\right)+l_{ki}^{*}\right], \quad i=k\end{array}\right.$$
  

$$\overline{w}_i^L=\frac{1}{n} \sum_{i \in N} w_i^L(k) \quad \overline{w}_i^U=\frac{1}{n} \sum_{i \in N} w_i^U(k)$$  

この結果は，正規性を満たしている．

***
## eAMRw/c, gAMRw/c, lAMRw/c (プログラム上はAMR-kaiと表記)

eAMRw/c. gAMRw/c, lAMRw/cの**Phase 4**を以下のように変更する：  
**Phase 4**  
Phase 3 で導かれた結果は$n$個の弱モデル分だけ考えられるため．以下のように結果を統合する：   
$$\left\{\begin{array}{l}w_i(k)=\left[\mu_{k}^* w_{ki}-l_{ki}^{*},\:\: \mu_{k}^* w_{ki}+l_{ki}^{*}\right], \quad i \in N\backslash k \\ w_i(k)=\left[\left(1-\mu_{k}^*\right)-l_{ki}^{*},\:\:\left(1-\mu_{k}^*\right)+l_{ki}^{*}\right], \quad i=k\end{array}\right.$$
  

$$\overline{w}_i^L=\frac{1}{n} \sum_{i \in N} w_i^L(k) \quad \overline{w}_i^U=\frac{1}{n} \sum_{i \in N} w_i^U(k)$$  

この結果は，正規性を満たしている．

***
***
## E-MMRw，G-MMRw (プログラム上はMMR-Wと表記)  
これは従来法である．各プログラムの動作チェックのためにjulia上でも実装した．

***
## E-AMRw，G-AMRw (プログラム上はAMR-Wと表記)  
これは従来法である．各プログラムの動作チェックのためにjulia上でも実装した．